# **Customer Lifetime Value Prediction**

## **Importing Files**

In [3]:
from google.colab import files
uploaded = files.upload()

Saving customer_segmentation.csv to customer_segmentation.csv


## **Identifying Dataset Properties**

In [4]:
import pandas as pd

In [8]:
df = pd.read_csv('customer_segmentation.csv', encoding='latin1')

In [9]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [10]:
df.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12/9/2011 12:50,4.95,12680.0,France


In [11]:
df.shape

(541909, 8)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [13]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [14]:
df.isnull()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
541904,False,False,False,False,False,False,False,False
541905,False,False,False,False,False,False,False,False
541906,False,False,False,False,False,False,False,False
541907,False,False,False,False,False,False,False,False


In [15]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [16]:
df.isnull().sum().sum()

136534

## **Data Preprocessing**

In [19]:
most_frequent_description = df['Description'].mode()[0]
df['Description'].fillna(most_frequent_description, inplace=True)

In [20]:
df['CustomerID'].fillna(-1, inplace=True)

In [21]:
df.isnull().sum().sum()

0

In [22]:
for i in range(len(df.columns)):
  print(df.columns[i])
  print(df[df.columns[i]].unique())
  print('## ' * 20)

InvoiceNo
['536365' '536366' '536367' ... '581585' '581586' '581587']
## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## 
StockCode
['85123A' '71053' '84406B' ... '90214Z' '90089' '23843']
## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## 
Description
['WHITE HANGING HEART T-LIGHT HOLDER' 'WHITE METAL LANTERN'
 'CREAM CUPID HEARTS COAT HANGER' ... 'PINK CRYSTAL SKULL PHONE CHARM'
 'CREAM HANGING HEART T-LIGHT HOLDER' 'PAPER CRAFT , LITTLE BIRDIE']
## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## 
Quantity
[     6      8      2     32      3      4     24     12     48     18
     20     36     80     64     10    120     96     23      5      1
     -1     50     40    100    192    432    144    288    -12    -24
     16      9    128     25     30     28      7     72    200    600
    480     -6     14     -2     -4     -5     -7     -3     11     70
    252     60    216    384     27    108     52  -9360     75    270
     42    240     90    320     

In [23]:
from sklearn.preprocessing import LabelEncoder

In [24]:
le = LabelEncoder()
df['Description'] = le.fit_transform(df['Description'])
df['Country'] = le.fit_transform(df['Country'])

### **Converting object type columns values to numeric type.**

In [25]:
for i in range(len(df.columns)):
  print(df.columns[i])
  print(df[df.columns[i]].unique())
  print('## ' * 20)

InvoiceNo
['536365' '536366' '536367' ... '581585' '581586' '581587']
## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## 
StockCode
['85123A' '71053' '84406B' ... '90214Z' '90089' '23843']
## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## 
Description
[3716 3724  861 ... 2456  864 2329]
## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## 
Quantity
[     6      8      2     32      3      4     24     12     48     18
     20     36     80     64     10    120     96     23      5      1
     -1     50     40    100    192    432    144    288    -12    -24
     16      9    128     25     30     28      7     72    200    600
    480     -6     14     -2     -4     -5     -7     -3     11     70
    252     60    216    384     27    108     52  -9360     75    270
     42    240     90    320     17   1824    204     69    -36   -192
   -144    160   2880   1400     19     39    -48    -50     56     13
   1440     -8     15    720    -20    156    324     4

In [27]:
from sklearn.preprocessing import MinMaxScaler

In [60]:
scaler = MinMaxScaler()
model = scaler.fit(df[['UnitPrice']])
df[['UnitPrice']] = model.transform(df[['UnitPrice']])

In [61]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,3716,0.500037,12/1/2010 8:26,0.000065,17850.0,35
1,536365,71053,3724,0.500037,12/1/2010 8:26,0.000087,17850.0,35
2,536365,84406B,861,0.500049,12/1/2010 8:26,0.000071,17850.0,35
3,536365,84029G,1813,0.500037,12/1/2010 8:26,0.000087,17850.0,35
4,536365,84029E,2776,0.500037,12/1/2010 8:26,0.000087,17850.0,35


In [62]:
df.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,2263,0.500074,12/9/2011 12:50,0.000022,12680.0,13
541905,581587,22899,695,0.500037,12/9/2011 12:50,0.000054,12680.0,13
541906,581587,23254,701,0.500025,12/9/2011 12:50,0.000106,12680.0,13
541907,581587,23255,700,0.500025,12/9/2011 12:50,0.000106,12680.0,13
541908,581587,22138,271,0.500019,12/9/2011 12:50,0.000127,12680.0,13


In [31]:
import matplotlib.pyplot as plt
import seaborn as sns

## **Model Training**

In [63]:
X = df[['Quantity', 'Description']]
y = df['UnitPrice']

In [34]:
from sklearn.model_selection import train_test_split

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
from sklearn.linear_model import LogisticRegression

In [65]:
model = LogisticRegression()

In [67]:
print(y_train.dtype)

float64


In [68]:
y_train = y_train.astype('category')

In [ ]:
model.fit(X_train, y_train)

## **Model Deployment**

In [ ]:
y_pred = model.predict(X_test)

In [71]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {rmse:.2f}')
print(f'R-squared (R2): {r2:.2f}')